# Loading models from checkpoints demo

I have a local ckpt file, and I want to load it and run inference on it.

In [1]:
from kirby.utils.train_wrapper import TrainWrapper
wrapper = TrainWrapper.load_from_checkpoint("../scripts/logs/lightning_logs/version_30/checkpoints/last.ckpt")

[14:38:21] INFO     (੭｡╹▿╹｡)੭ Poyo!

[14:38:24] INFO     Created a temporary directory at /tmp/tmpbc1inpkl

           INFO     Writing /tmp/tmpbc1inpkl/_remote_module_non_scriptable.py

[14:38:30] WARNING  xformers not installed. Won't use memory-efficient attention.

/home/mila/p/patrick.mineault/.conda/envs/poyo/lib/python3.9/site-packages/lightning/pytorch/utilities/parsing.py:196: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  rank_zero_warn(


In [12]:
wrapper.model

PerceiverNM(
  (unit_emb): Embedding(226, 64)
  (spike_type_emb): Embedding(4, 64)
  (task_emb): Embedding(64, 64)
  (latent_emb): Embedding(16, 64)
  (rotary_emb): RotaryEmbedding()
  (dropout): Dropout(p=0.4, inplace=False)
  (enc_atn): RotaryCrossAttention(
    (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (norm_context): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (to_q): Linear(in_features=64, out_features=128, bias=False)
    (to_kv): Linear(in_features=64, out_features=256, bias=False)
    (to_out): Linear(in_features=128, out_features=64, bias=True)
  )
  (enc_ffn): Sequential(
    (0): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (1): FeedForward(
      (net): Sequential(
        (0): Linear(in_features=64, out_features=512, bias=True)
        (1): GEGLU()
        (2): Dropout(p=0.2, inplace=False)
        (3): Linear(in_features=256, out_features=64, bias=True)
      )
    )
  )
  (proc_layers): ModuleList(
    (0-11): 12 x ModuleLi

In [28]:
wrapper.model.dec_ffn[0].weight.mean()

tensor(0.9622, device='cuda:0', grad_fn=<MeanBackward0>)

In [30]:
import torch
ckpt_raw = torch.load("../scripts/logs/lightning_logs/version_30/checkpoints/last.ckpt")
ckpt_raw['state_dict']["model.dec_ffn.0.weight"].mean()

tensor(0.9622, device='cuda:0')

# Load from wandb

I've uploaded the checkpoint to wandb via:

```wandb artifact put logs/lightning_logs/version_30/checkpoints/last.ckpt -t model -n single_session```

Instantiate this model from the cloud.

In [33]:
import wandb

run = wandb.init(project="poyo", entity="neuro-galaxy", )

artifact = run.use_artifact("neuro-galaxy/poyo/single_session:latest")
path = artifact.download()

wandb:   1 of 1 files downloaded.  


Now that the model is downloaded, we can easily recover it using the same mechanism.

In [35]:
wrapper_cloud = TrainWrapper.load_from_checkpoint(f"{path}/last.ckpt")

/home/mila/p/patrick.mineault/.conda/envs/poyo/lib/python3.9/site-packages/lightning/pytorch/utilities/parsing.py:196: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  rank_zero_warn(


In [36]:
wrapper_cloud.model

PerceiverNM(
  (unit_emb): Embedding(226, 64)
  (spike_type_emb): Embedding(4, 64)
  (task_emb): Embedding(64, 64)
  (latent_emb): Embedding(16, 64)
  (rotary_emb): RotaryEmbedding()
  (dropout): Dropout(p=0.4, inplace=False)
  (enc_atn): RotaryCrossAttention(
    (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (norm_context): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (to_q): Linear(in_features=64, out_features=128, bias=False)
    (to_kv): Linear(in_features=64, out_features=256, bias=False)
    (to_out): Linear(in_features=128, out_features=64, bias=True)
  )
  (enc_ffn): Sequential(
    (0): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (1): FeedForward(
      (net): Sequential(
        (0): Linear(in_features=64, out_features=512, bias=True)
        (1): GEGLU()
        (2): Dropout(p=0.2, inplace=False)
        (3): Linear(in_features=256, out_features=64, bias=True)
      )
    )
  )
  (proc_layers): ModuleList(
    (0-11): 12 x ModuleLi